# TRI30603

In [ ]:
import pandas as pd, requests
from io import BytesIO
tabella = 'TRI30602'
file = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
result = requests.get(file)
date_column = ['DATA_OSS']
data = pd.read_csv(BytesIO(result.content),compression='zip',header=0,sep=';',quotechar='"',encoding='utf-8',decimal=',',parse_dates=['DATA_OSS'],dayfirst=False)
data['DATA_OSS'] = pd.to_datetime(data['DATA_OSS'])# .dt.date


In [8]:
import pandas as pd
import requests
from io import BytesIO

tabelle = ['TRI30601', 'TRI30602', 'TRI30603', 'TRI30604', 'TRI30605', 'TRI30606']
df_list = []

for tabella in tabelle:
    url = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
    print(f"Scarico: {tabella}")
    result = requests.get(url)

    # Leggi i dati SENZA inferenza automatica dei tipi (dtype=str forza tutto come stringa)
    df = pd.read_csv(BytesIO(result.content),
                     compression='zip',
                     header=0,
                     sep=';',
                     quotechar='"',
                     encoding='utf-8',
                     decimal=',',
                     dtype=str)  # ← impedisce perdita zeri

    # Conversione controllata
    df['DATA_OSS'] = pd.to_datetime(df['DATA_OSS'], errors='coerce')
    df['VALORE'] = pd.to_numeric(df['VALORE'], errors='coerce')

    # Aggiungi la colonna della tabella
    df['TABELLA'] = tabella

    df_list.append(df)

# Unione
df_all = pd.concat(df_list, ignore_index=True)

# Controllo finale
print(df_all.dtypes)
print(df_all.head())


Scarico: TRI30601
Scarico: TRI30602
Scarico: TRI30603
Scarico: TRI30604
Scarico: TRI30605
Scarico: TRI30606
CLASSE_UTILIZZ            object
DATA_OSS          datetime64[ns]
ENTE_SEGN                 object
FENEC                     object
SET_CTP                   object
VALORE                   float64
STATUS                    object
TABELLA                   object
SEDELEG_SOGG              object
ATECO_CTP                 object
dtype: object
  CLASSE_UTILIZZ   DATA_OSS ENTE_SEGN  ...   TABELLA SEDELEG_SOGG  ATECO_CTP
0           1006 2025-03-31   3691030  ...  TRI30601          NaN        NaN
1           1005 2025-03-31   3691030  ...  TRI30601          NaN        NaN
2           9904 2025-03-31   3691030  ...  TRI30601          NaN        NaN
3           9904 2025-03-31   3691030  ...  TRI30601          NaN        NaN
4           1005 2025-03-31   3691030  ...  TRI30601          NaN        NaN

[5 rows x 10 columns]


In [9]:
data.columns

Index(['DATA_OSS', 'ENTE_SEGN', 'FENEC', 'SEDELEG_SOGG', 'SET_CTP', 'VALORE',
       'STATUS'],
      dtype='object')

In [ ]:
['DATA_OSS', 'ENTE_SEGN', 'FENEC', 'SEDELEG_SOGG', 'SET_CTP', 'VALORE','STATUS']
['DATA_OSS', 'ENTE_SEGN', 'FENEC', 'SEDELEG_SOGG', 'SET_CTP', 'VALORE','STATUS']


In [13]:
df_all.columns

Index(['CLASSE_UTILIZZ', 'DATA_OSS', 'ENTE_SEGN', 'FENEC', 'SET_CTP', 'VALORE',
       'STATUS', 'TABELLA', 'SEDELEG_SOGG', 'ATECO_CTP'],
      dtype='object')

In [11]:
df_all

,CLASSE_UTILIZZ,DATA_OSS,ENTE_SEGN,FENEC,SET_CTP,VALORE,STATUS,TABELLA,SEDELEG_SOGG,ATECO_CTP
0,1006,2025-03-31,3691030,35110163,S15BI1,NaN,NaN,TRI30601,NaN,NaN
1,1005,2025-03-31,3691030,35110363,S11,NaN,NaN,TRI30601,NaN,NaN
2,9904,2025-03-31,3691030,35110363,S14BI4,NaN,NaN,TRI30601,NaN,NaN
3,9904,2025-03-31,3691030,35110163,S12BI7,NaN,NaN,TRI30601,NaN,NaN
4,1005,2025-03-31,3691030,351151433,600,888061517.0,NaN,TRI30601,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1731721,1005,2006-06-30,3691030,351151433,SBI25,2543187.0,NaN,TRI30606,ITF,28
1731722,9904,2006-06-30,3691030,351151441,SBI25,769.0,NaN,TRI30606,IT,1000074
1731723,9904,2006-06-30,3691030,351151441,SBI25,484.0,NaN,TRI30606,IT,1000061
1731724,1005,2006-06-30,3691030,351151441,SBI25,61.0,NaN,TRI30606,ITG,I


In [14]:
import pandas as pd
import requests
from io import BytesIO

tabelle = ['TRI30601', 'TRI30602', 'TRI30603', 'TRI30604', 'TRI30605', 'TRI30606']
df_dict = {}

# Scarica ogni tabella e salvala in un dizionario
for tabella in tabelle:
    url = f'https://a2a.bancaditalia.it/infostat/dataservices/export/IT/CSV/DATA/CUBE/BANKITALIA/DIFF/{tabella}'
    print(f"Scarico: {tabella}")
    result = requests.get(url)

    df = pd.read_csv(BytesIO(result.content),
                     compression='zip',
                     header=0,
                     sep=';',
                     quotechar='"',
                     encoding='utf-8',
                     decimal=',',
                     dtype=str)

    # Parsing colonne note
    df['DATA_OSS'] = pd.to_datetime(df['DATA_OSS'], errors='coerce')
    df['VALORE'] = pd.to_numeric(df['VALORE'], errors='coerce')
    df['TABELLA'] = tabella

    df_dict[tabella] = df  # salva nel dizionario

# Mappa: nome tabella → set colonne
columns_per_df = {name: set(df.columns) for name, df in df_dict.items()}

# Colonne comuni a tutte
colonne_comuni = set.intersection(*columns_per_df.values())

# Colonne totali
colonne_totali = set.union(*columns_per_df.values())

# Colonne non comuni: per ciascuna tabella, mostra solo quelle non condivise con tutte
colonne_non_comuni = {
    name: list(cols - colonne_comuni) for name, cols in columns_per_df.items()
}

# Output
print("\n📌 Colonne comuni a tutte le tabelle:")
print(sorted(colonne_comuni))

print("\n🧩 Colonne specifiche di ogni tabella:")
for name, cols in colonne_non_comuni.items():
    print(f"{name}: {sorted(cols)}")


Scarico: TRI30601
Scarico: TRI30602
Scarico: TRI30603
Scarico: TRI30604
Scarico: TRI30605
Scarico: TRI30606

📌 Colonne comuni a tutte le tabelle:
['DATA_OSS', 'ENTE_SEGN', 'FENEC', 'SET_CTP', 'STATUS', 'TABELLA', 'VALORE']

🧩 Colonne specifiche di ogni tabella:
TRI30601: ['CLASSE_UTILIZZ']
TRI30602: ['SEDELEG_SOGG']
TRI30603: ['SEDELEG_SOGG']
TRI30604: ['CLASSE_UTILIZZ', 'SEDELEG_SOGG']
TRI30605: ['ATECO_CTP', 'SEDELEG_SOGG']
TRI30606: ['ATECO_CTP', 'CLASSE_UTILIZZ', 'SEDELEG_SOGG']


In [15]:
import pandas as pd

# Unione di tutti i DataFrame in uno solo
df_all = pd.concat(df_dict.values(), ignore_index=True)

# Identifica le colonne fisse (non dimensionali)
colonne_fisse = ['DATA_OSS', 'VALORE', 'TABELLA', 'ENTE_SEGN', 'FENEC', 'SET_CTP', 'STATUS']

# Tutte le altre sono dimensioni da trasformare
colonne_dim = [col for col in df_all.columns if col not in colonne_fisse]

# Melt (wide → long)
df_lungo = df_all.melt(
    id_vars=colonne_fisse,
    value_vars=colonne_dim,
    var_name="NOME_DIM",
    value_name="VALORE_DIM"
)

# Ordina per leggibilità
df_lungo = df_lungo.sort_values(by=["TABELLA", "DATA_OSS", "NOME_DIM"])

# Controllo finale
print(df_lungo.head(10))


          DATA_OSS        VALORE   TABELLA  ... STATUS   NOME_DIM VALORE_DIM
3476052 2006-06-30  5.382917e+10  TRI30601  ...    NaN  ATECO_CTP        NaN
3476053 2006-06-30  2.025000e+03  TRI30601  ...    NaN  ATECO_CTP        NaN
3476054 2006-06-30  3.436000e+03  TRI30601  ...    NaN  ATECO_CTP        NaN
3476055 2006-06-30           NaN  TRI30601  ...    NaN  ATECO_CTP        NaN
3476056 2006-06-30           NaN  TRI30601  ...    NaN  ATECO_CTP        NaN
3476057 2006-06-30           NaN  TRI30601  ...    NaN  ATECO_CTP        NaN
3476058 2006-06-30           NaN  TRI30601  ...    NaN  ATECO_CTP        NaN
3476059 2006-06-30           NaN  TRI30601  ...    NaN  ATECO_CTP        NaN
3476060 2006-06-30           NaN  TRI30601  ...    NaN  ATECO_CTP        NaN
3476061 2006-06-30  1.993984e+06  TRI30601  ...    NaN  ATECO_CTP        NaN

[10 rows x 9 columns]


In [17]:
df_lungo[df_lungo['DATA_OSS'] == '2025-03-31']

,DATA_OSS,VALORE,TABELLA,ENTE_SEGN,FENEC,SET_CTP,STATUS,NOME_DIM,VALORE_DIM
3463452,2025-03-31,NaN,TRI30601,3691030,35110163,S15BI1,NaN,ATECO_CTP,NaN
3463453,2025-03-31,NaN,TRI30601,3691030,35110363,S11,NaN,ATECO_CTP,NaN
3463454,2025-03-31,NaN,TRI30601,3691030,35110363,S14BI4,NaN,ATECO_CTP,NaN
3463455,2025-03-31,NaN,TRI30601,3691030,35110163,S12BI7,NaN,ATECO_CTP,NaN
3463456,2025-03-31,888061517.0,TRI30601,3691030,351151433,600,NaN,ATECO_CTP,NaN
...,...,...,...,...,...,...,...,...,...
3122417,2025-03-31,44.0,TRI30606,3691030,351151441,SBI25,NaN,SEDELEG_SOGG,ITC
3122418,2025-03-31,44.0,TRI30606,3691030,351151441,SBI25,NaN,SEDELEG_SOGG,ITF
3122419,2025-03-31,915508162.0,TRI30606,3691030,351152133,SBI25,NaN,SEDELEG_SOGG,ITI
3122420,2025-03-31,79293374.0,TRI30606,3691030,351152133,SBI25,NaN,SEDELEG_SOGG,ITC
